# RAS Commander Project Initialization

This notebook demonstrates how to initialize and work with HEC-RAS projects using the `ras-commander` library. You'll learn how to:

1. Set up and configure the RAS Commander environment
2. Download and extract example HEC-RAS projects
3. Initialize HEC-RAS projects using the global `ras` object
4. Initialize multiple HEC-RAS projects using custom RAS objects
5. Access various project components (plans, geometries, flows, boundaries)

## Key Concepts

- **RasPrj Objects**: Represent HEC-RAS projects with access to plans, geometries, flows, etc.
- **Global `ras` object**: A singleton instance for simple, single-project scripts
- **Custom RAS Objects**: Independent instances for multi-project workflows
- **Project Initialization**: Process of connecting to HEC-RAS projects
- **Project Components**: Structured access to plans, geometries, and flow files

Let's start by importing the necessary libraries.

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [ ]:
# 1. Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

# 2. Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import os
from pathlib import Path
import pandas as pd
from IPython import display

In [ ]:
# This cell will try to import the pip package, if it fails it will 
# add the parent directory to the Python path and try to import again
# This assumes you are working in a subfolder of the ras-commander repository
# This allows a user's revisions to be tested locally without installing the package

import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import *
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    rascmdr_directory = current_file.parent
    sys.path.append(str(rascmdr_directory))
    print("Loading ras-commander from local dev copy")
    # Now try to import again
    from ras_commander import *
print("ras_commander imported successfully")


## RAS Commander: Core Concepts

RAS Commander is a Python library that provides tools for automating HEC-RAS tasks. It's built with several key design principles:

1. **Project-Centric Architecture**: Everything revolves around HEC-RAS projects
2. **Two RAS Object Approaches**:
   - **Global `ras` Object**: A singleton for simple scripts
   - **Custom RAS Objects**: Multiple ras project instances for complex workflows
3. **Comprehensive Project Representation**: Each RAS object includes DataFrames for plans, geometries, flows, and boundaries
4. **Logging**: Built-in logging to track operations and debug issues
5. **HDF Support**: Specialized functions for HDF file access (plan results, geometry, etc.)

Let's explore these concepts in practice.

## Downloading Example HEC-RAS Projects

RAS Commander includes a utility to download and extract example HEC-RAS projects. These are useful for learning and testing:

In [ ]:
# Extract specific projects we'll use in this tutorial
# This will download them if not present and extract them to the example_projects folder
extracted_paths = RasExamples.extract_project(["Balde Eagle Creek", "BaldEagleCrkMulti2D", "Muncie"])
print(extracted_paths)

# Get Paths for Extracted Example Projects

In [ ]:
# Get the parent directory of the first extracted path as our examples directory
examples_dir = extracted_paths[0].parent
print(f"Examples directory: {examples_dir}")


# Define paths to the extracted projects
bald_eagle_path = examples_dir / "Balde Eagle Creek"
multi_2d_path = examples_dir / "BaldEagleCrkMulti2D"
muncie_path = examples_dir / "Muncie"

# Verify the paths exist
for path in [bald_eagle_path, multi_2d_path, muncie_path]:
    print(f"Path {path} exists: {path.exists()}")

## Utility Function to Print RAS Object Data

Let's create a utility function to help us explore the contents of RAS objects:

In [ ]:
def print_ras_object_data(ras_obj, project_name):
    """Prints comprehensive information about a RAS object"""
    print(f"\n{project_name} Data:")
    print("=" * 50)
    print(f"Project Name: {ras_obj.get_project_name()}")
    print(f"Project Folder: {ras_obj.project_folder}")
    print(f"PRJ File: {ras_obj.prj_file}")
    print(f"HEC-RAS Executable Path: {ras_obj.ras_exe_path}")
    
    print("\nPlan Files DataFrame (ras.plan_df):")
    with pd.option_context('display.max_columns', None):
        display.display(ras_obj.plan_df)
    
    print("\nSteady Flow Files DataFrame:")
    display.display(ras_obj.flow_df)
    
    print("\nUnsteady Flow Files DataFrame (ras.unsteady_df):")
    display.display(ras_obj.unsteady_df)
    
    print("\nGeometry Files DataFrame (ras.geom_df):")
    display.display(ras_obj.geom_df)
    
    print("\nHDF Entries DataFrame (ras.get_hdf_entries()):")
    display.display(ras_obj.get_hdf_entries())
    
    print("\nBoundary Conditions DataFrame (ras.boundaries_df):")
    display.display(ras_obj.boundaries_df)

## Approach 1: Using the Global `ras` Object

The global `ras` object is a singleton instance that persists throughout your script. It's ideal for simple scripts working with a single project.

Key characteristics:
- It's available as `ras` immediately after import
- It's initialized via `init_ras_project()` without saving the return value
- It provides access to all project data through the global `ras` variable
- It's simple to use but can be problematic in complex scenarios

Let's initialize it with the Bald Eagle Creek project:

In [ ]:
# Initialize the global ras object with Bald Eagle Creek project
# Note: This updates the global 'ras' object visible throughout the script
# Parameters:
#   - project_folder: Path to the HEC-RAS project folder (required)
#   - ras_version: HEC-RAS version (e.g. "6.5") or path to Ras.exe (required first time)

init_ras_project(bald_eagle_path, "6.5")
print(f"The global 'ras' object is now initialized with the {ras.project_name} project")

In [ ]:
# Find the .prj file in the Bald Eagle Creek folder
prj_file = list(bald_eagle_path.glob("*.prj"))[0]
print(f"Found .prj file: {prj_file}")
print(f"File name: {prj_file.name}\n")

# Initialize using the .prj file path directly (NEW FEATURE!)
# This works exactly the same as passing the folder path
init_ras_project(prj_file, "6.5")
print(f"Successfully initialized project using .prj file path!")
print(f"Project name: {ras.project_name}")
print(f"Project folder: {ras.project_folder}")
print(f"PRJ file: {ras.prj_file}")

# Verify both methods produce identical results
print(f"\n✓ Both folder path and .prj file path initialization methods produce the same result!")
print(f"✓ The project folder is automatically extracted from the .prj file's parent directory")

### Alternative: Initialize Using .prj File Path

**New Feature**: You can now initialize a project by providing the direct path to a `.prj` file instead of the project folder. This is especially useful when:
- Working with file selection dialogs (which return file paths)
- Using configuration files that store specific .prj file paths
- Working with folders that contain multiple projects
- Building command-line tools that accept file paths

The function automatically:
1. Validates that the file has a `.prj` extension
2. Verifies the file contains "**Proj Title=**" to confirm it's a HEC-RAS project file (not a plan file)
3. Extracts the parent folder and uses it as the project folder
4. Optimizes initialization by passing the .prj file directly (avoids re-searching)

Let's see this in action:

In [ ]:
# Explore the global ras object with our utility function
print_ras_object_data(ras, "Global RAS Object (Bald Eagle Creek)")

### Understanding the RAS Object Structure

Each RAS object contains several important components:

1. **Project Metadata**:
   - `project_name`: Name of the HEC-RAS project
   - `project_folder`: Directory containing project files
   - `prj_file`: Path to the main .prj file
   - `ras_exe_path`: Path to the HEC-RAS executable

2. **Project DataFrames**:
   - `plan_df`: Information about all plan files (.p*)
   - `flow_df`: Information about all steady flow files (.f*)
   - `unsteady_df`: Information about all unsteady flow files (.u*)
   - `geom_df`: Information about all geometry files (.g*)
   - `boundaries_df`: Information about all boundary conditions

3. **Methods for Data Access**:
   - `get_plan_entries()`: Get plan file information
   - `get_flow_entries()`: Get flow file information
   - `get_unsteady_entries()`: Get unsteady flow file information 
   - `get_geom_entries()`: Get geometry file information
   - `get_hdf_entries()`: Get HDF file paths for result files
   - `get_boundary_conditions()`: Get boundary condition details

Let's see how to access specific information from these components:

In [ ]:
# Get the first plan's details
if not ras.plan_df.empty:
    first_plan = ras.plan_df.iloc[0]
    print(f"First plan number: {first_plan['plan_number']}")
    print(f"Plan path: {first_plan['full_path']}")
    
    # Get the geometry file for this plan
    geom_id = first_plan.get('Geom File', '').replace('g', '')
    if geom_id:
        geom_info = ras.geom_df[ras.geom_df['geom_number'] == geom_id]
        if not geom_info.empty:
            print(f"Geometry file: {geom_info.iloc[0]['full_path']}")
    
    # Get the HDF results file for this plan (if exists)
    if 'HDF_Results_Path' in first_plan and first_plan['HDF_Results_Path']:
        print(f"Results file: {first_plan['HDF_Results_Path']}")
else:
    print("No plans found in the project.")

### Working with Boundary Conditions

Boundary conditions define the inputs and outputs of your model. Let's see how to access boundary condition information:

In [ ]:
# View the boundary conditions DataFrame
ras.boundaries_df 

## Approach 2: Using Custom RAS Objects

For more complex scripts or when working with multiple projects, it's better to create and use separate RAS objects. This approach:

- Creates independent RAS objects for each project
- Avoids overwriting the global `ras` object
- Provides clearer separation between projects
- Allows working with multiple projects simultaneously
- Requires saving the return value from `init_ras_project()`

Let's initialize multiple projects with custom RAS objects:

In [ ]:
# Initialize multiple project instances with custom RAS objects
# Note: This also updates the global 'ras' object each time, but we'll use the custom instances
# Parameters remain the same as before
multi_2d_project = RasPrj()
init_ras_project(multi_2d_path, "6.5", ras_object=multi_2d_project)
print(f"\nMulti2D project initialized with its own RAS object")

muncie_project = RasPrj()
init_ras_project(muncie_path, "6.5", ras_object=muncie_project)
print(f"\nMuncie project initialized with its own RAS object")

# Note that the global 'ras' object now points to the Muncie project
# The global 'ras' object gets overwritten every time a project is initialized ,
print(f"\nGlobal 'ras' object now points to: {ras.project_name} since it was the last one initialized.  Avoid the global object when using multiple projects.")

### Exploring Multiple Projects

Now we have three RAS objects:
- `multi_2d_project`: Our custom object for the Multi2D project
- `muncie_project`: Our custom object for the Muncie project
- `ras`: The global object (which now points to Muncie)

Let's examine the Multi2D project:

In [ ]:
display.display(multi_2d_project.plan_df)

In [ ]:
# Examine the Multi2D project
print_ras_object_data(multi_2d_project, "Multi2D Project")

In [ ]:
# Examine the Muncie project
print_ras_object_data(muncie_project, "Muncie Project")

### Comparing Projects

Let's compare some key metrics of the two projects:

In [ ]:
# Create a comparison table of the two projects
comparison_data = {
    'Project Name': [multi_2d_project.project_name, muncie_project.project_name],
    'Number of Plans': [len(multi_2d_project.plan_df), len(muncie_project.plan_df)],
    'Number of Geometries': [len(multi_2d_project.geom_df), len(muncie_project.geom_df)],
    'Number of Flow Files': [len(multi_2d_project.flow_df), len(muncie_project.flow_df)],
    'Number of Unsteady Files': [len(multi_2d_project.unsteady_df), len(muncie_project.unsteady_df)],
    'Number of Boundary Conditions': [len(multi_2d_project.boundaries_df) if hasattr(multi_2d_project, 'boundaries_df') else 0, 
                                     len(muncie_project.boundaries_df) if hasattr(muncie_project, 'boundaries_df') else 0],
    'HDF Results Available': [len(multi_2d_project.get_hdf_entries()) > 0, len(muncie_project.get_hdf_entries()) > 0]
}

comparison_df = pd.DataFrame(comparison_data)
display.display(comparison_df)

## RAS Commander: Best Practices

After exploring both approaches, here are some best practices for using RAS Commander:

1. **Choose Your Approach Based on Complexity**:
   - **Simple Scripts** (one project): Use the global `ras` object
   - **Complex Scripts** (multiple projects): Use custom RAS objects

2. **Be Consistent**:
   - Don't mix global and custom approaches in the same script
   - Use descriptive names for custom RAS objects

3. **Working with Project Files**:
   - Access project files through the RAS object's DataFrames
   - Use helper functions like `get_plan_path()` to resolve paths

4. **Error Handling**:
   - Always check for empty DataFrames before accessing their contents
   - Use the built-in logging to track operations

5. **Performance Considerations**:
   - For large projects, consider using the HDF classes directly
   - Cache results of expensive operations when possible

## Summary of Key Functions

- `init_ras_project(project_folder, ras_version)`: Initialize a RAS project
- `RasExamples().extract_project(project_name)`: Extract example projects
- `RasPrj.get_project_name()`: Get the name of the project
- `RasPrj.get_plan_entries()`: Get plan file information
- `RasPrj.get_flow_entries()`: Get flow file information
- `RasPrj.get_unsteady_entries()`: Get unsteady flow file information
- `RasPrj.get_geom_entries()`: Get geometry file information
- `RasPrj.get_hdf_entries()`: Get HDF result file information
- `RasPrj.get_boundary_conditions()`: Get boundary condition details
- `RasPlan.get_plan_path(plan_number)`: Get the path to a plan file
- `RasPlan.get_geom_path(geom_number)`: Get the path to a geometry file
- `RasPlan.get_flow_path(flow_number)`: Get the path to a flow file
- `RasPlan.get_unsteady_path(unsteady_number)`: Get the path to an unsteady flow file

## Next Steps

Now that you understand the basics of project initialization in RAS Commander, you can explore more advanced topics:

1. Working with HDF files for result analysis
2. Modifying plan, geometry, and flow files
3. Running HEC-RAS simulations
4. Extracting and visualizing results
5. Automating model calibration

These topics are covered in other examples and notebooks in the RAS Commander documentation.